Introductie

P1

P2

P3

P4

## De invloed van klimaat en geografie op voedselproductie

Niet elk land beschikt over dezelfde natuurlijke omstandigheden om voedsel te produceren. Factoren zoals neerslag, temperatuur en klimaatzones bepalen welke gewassen efficiënt kunnen groeien. In landen met een ongunstig klimaat moeten gewassen onder intensievere en vaak minder duurzame omstandigheden worden verbouwd.
Daarnaast speelt de geografische ligging een rol in hoe voedsel wordt verhandeld: landen die ver van exportmarkten liggen, hebben hogere transportkosten en extra uitstoot, wat hun voedselsysteem minder duurzaam maakt.
In dit perspectief onderzoeken we hoe deze geografische factoren — natuurlijke omstandigheden en ligging ten opzichte van wereldmarkten — bijdragen aan verschillen in de duurzaamheid van nationale voedselsystemen.

#### Klimaatzones en natuurlijke omstandigheden beïnvloeden duurzaamheid
Landen verschillen sterk in klimaat. Tropische regio’s kennen bijvoorbeeld hoge temperaturen en regelmatige regenval, terwijl droge of koude gebieden kampen met beperkte groeiseizoenen en lagere landbouwopbrengsten per hectare.
In warmere, vruchtbare klimaatzones kunnen gewassen natuurlijk groeien, met weinig extra input. In koudere of drogere gebieden zijn irrigatie, verwarming, kunstmest of import van hulpbronnen nodig om dezelfde opbrengst te halen. Deze extra inputs verhogen de uitstoot per kilogram geproduceerd voedsel.
Daarom is het voedselsysteem in sommige landen per definitie minder duurzaam, simpelweg door hun ligging in een ongunstige klimaatzone.

In [39]:
import pandas as pd
import plotly.express as px

neerslag_df = pd.read_csv("datasets/API_AG.LND.PRCP.MM_DS2_en_csv_v2_4487.csv", skiprows=4)
emissies_crops_df = pd.read_csv("datasets/Emissions_crops_filtered.csv")
emissies_livestock_df = pd.read_csv("datasets/Emissions_livestock_filtered.csv")

neerslag_long = neerslag_df.melt(
    id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"],
    var_name="Year",
    value_name="Precipitation_mm"
)
neerslag_long = neerslag_long[["Country Name", "Year", "Precipitation_mm"]]
neerslag_long["Year"] = pd.to_numeric(neerslag_long["Year"], errors="coerce")
neerslag_long = neerslag_long.dropna(subset=["Year", "Precipitation_mm"])
neerslag_long["Country Name"] = neerslag_long["Country Name"].replace({
    "United States": "United States of America"
})

# ⬇️ Bereken totale uitstoot per land en jaar (gewassen + vee)
emissies_crops_total = emissies_crops_df.groupby(["Area", "Year"])["Value"].sum().reset_index()
emissies_livestock_total = emissies_livestock_df.groupby(["Area", "Year"])["Value"].sum().reset_index()

emissies_total = pd.merge(
    emissies_crops_total, emissies_livestock_total,
    on=["Area", "Year"], how="outer", suffixes=("_crops", "_livestock")
)
emissies_total["Emissions_total"] = (
    emissies_total["Value_crops"].fillna(0) + emissies_total["Value_livestock"].fillna(0)
)
emissies_total = emissies_total[["Area", "Year", "Emissions_total"]]
emissies_total.rename(columns={"Area": "Country Name"}, inplace=True)

# ⬇️ Merge emissies met neerslag
combined_df = pd.merge(neerslag_long, emissies_total, on=["Country Name", "Year"])

# ⬇️ Filter op jouw 8 landen
landen_selectie = [
    "Nigeria", "South Africa", "India", "Japan",
    "United States of America", "Brazil", "Indonesia"
]
combined_df = combined_df[combined_df["Country Name"].isin(landen_selectie)]

# ⬇️ Maak scatterplot met animatie
fig = px.scatter(
    combined_df,
    x="Precipitation_mm",
    y="Emissions_total",
    animation_frame="Year",
    color="Country Name",
    text="Country Name",
    hover_name="Country Name",
    title="Neerslag vs Landbouwuitstoot per Jaar (8 landen)",
    labels={
        "Precipitation_mm": "Neerslag (mm/jaar)",
        "Emissions_total": "Totale landbouwuitstoot (Mt CO₂e)"
    },
    log_y=False
)

fig.update_traces(
    textposition='top center',
    marker=dict(size=14)
)

fig.show()


In [ ]:
import pandas as pd
import plotly.express as px

# Laad de datasets
neerslag_df = pd.read_csv("datasets/API_AG.LND.PRCP.MM_DS2_en_csv_v2_4487.csv", skiprows=4)
productie_df = pd.read_csv("datasets/Production_Crops_Livestock_filtered.csv")

# Zet neerslagdata om naar lange vorm
neerslag_long = neerslag_df.melt(
    id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"],
    var_name="Year",
    value_name="Precipitation_mm"
)
neerslag_long = neerslag_long[["Country Name", "Year", "Precipitation_mm"]]
neerslag_long["Year"] = pd.to_numeric(neerslag_long["Year"], errors="coerce")
neerslag_long = neerslag_long.dropna(subset=["Year", "Precipitation_mm"])
neerslag_long["Country Name"] = neerslag_long["Country Name"].replace({
    "United States": "United States of America"
})

# Verwerk productiedata
productie_filtered = productie_df[productie_df["Element"] == "Production"]
productie_aggregated = productie_filtered.groupby(["Area", "Year"])["Value"].sum().reset_index()
productie_aggregated.rename(columns={"Area": "Country Name", "Value": "Production_tonnes"}, inplace=True)
productie_aggregated["Year"] = pd.to_numeric(productie_aggregated["Year"], errors="coerce")
productie_aggregated["Country Name"] = productie_aggregated["Country Name"].replace({
    "Netherlands (Kingdom of the)": "Netherlands"
})

# Merge datasets
productie_neerslag_df = pd.merge(neerslag_long, productie_aggregated, on=["Country Name", "Year"])

# Filter op de 7 landen
landen_selectie = [
    "Nigeria", "South Africa", "India", "Japan",
    "United States of America", "Brazil", "Indonesia"
]
productie_neerslag_df = productie_neerslag_df[productie_neerslag_df["Country Name"].isin(landen_selectie)]

# Maak scatterplot met animatie
fig = px.scatter(
    productie_neerslag_df,
    x="Precipitation_mm",
    y="Production_tonnes",
    animation_frame="Year",
    color="Country Name",
    text="Country Name",
    hover_name="Country Name",
    title="Voedselproductie vs Neerslag per Jaar (7 landen)",
    labels={
        "Precipitation_mm": "Neerslag (mm/jaar)",
        "Production_tonnes": "Totale voedselproductie (ton)"
    }
)

fig.update_traces(
    textposition='top center',
    marker=dict(size=14)
)

fig.show()


FileNotFoundError: [Errno 2] No such file or directory: 'API_AG.LND.PRCP.MM_DS2_en_csv_v2_4487.csv'